In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torchvision as tv
import torchvision.transforms as tf
#import numpy

batch = 32
model = tv.models.resnet34(pretrained = True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = tf.Compose([tf.Resize(256), tf.ToTensor(),
                        tf.Normalize(mean = [0.5, 0.5, 0.5], std = [0.225, 0.225, 0.225]), 
                        tf.CenterCrop(224)])

orgset = tv.datasets.ImageFolder('./simpsons_dataset', transform = transform)
n = len(orgset)
test_n = int(0.1 * n) 
train_n = n - test_n

classes = orgset.classes
train_set, test_set = torch.utils.data.random_split(orgset, [train_n, test_n])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch, shuffle=True, num_workers=4)

model.fc = nn.Linear(model.fc.in_features, 42, bias = True)
loss_f = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum=0.86)
model.to(device)

In [2]:
for epoch in range(5):
    run_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs  = model(inputs)
        loss = loss_f(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        run_loss += loss.item()

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (labels == predicted).sum().item()

    print('epoch: ', epoch, '\n', 'accuracy: ', 100*correct/total, '\n')

epoch:  0 
 accuracy:  89.44099378881988 

epoch:  1 
 accuracy:  92.73769708552317 

epoch:  2 
 accuracy:  93.7888198757764 

epoch:  3 
 accuracy:  94.83994266602963 

epoch:  4 
 accuracy:  94.696607740086 



In [4]:
torch.save({'train_indices': train_set.indices,
            'test_indices': test_set.indices}, 
            'split_indices.pth')


PATH = './my_model.pth'
torch.save(model.state_dict(), PATH)

In [5]:
with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (labels == predicted).sum().item()

print('accuracy: ', 100*correct/total, '\n')

accuracy:  95.31772575250837 

